In [1]:
print("aksih")

aksih


In [31]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

import pandas as pd
import json
import os
import traceback
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
from langchain.chat_models import ChatOpenAI

In [33]:
KEY = os.getenv("OPENAI_API_KEY")

In [34]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [35]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "4": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "5": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [36]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [37]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [38]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [39]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [40]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [41]:
review_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="review", verbose=True)

In [42]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [43]:
file_path=r"C:\Users\akish\MCQ_generator_LLM\data.txt"
file_path

'C:\\Users\\akish\\MCQ_generator_LLM\\data.txt'

In [44]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [45]:
print(TEXT)

The history of artificial intelligence (AI) began in antiquity, with myths, stories and rumors of artificial beings endowed with intelligence or consciousness by master craftsmen. The seeds of modern AI were planted by philosophers who attempted to describe the process of human thinking as the mechanical manipulation of symbols. This work culminated in the invention of the programmable digital computer in the 1940s, a machine based on the abstract essence of mathematical reasoning. This device and the ideas behind it inspired a handful of scientists to begin seriously discussing the possibility of building an electronic brain.

Alan Turing was the first person to carry out substantial research in the field that he called Machine Intelligence.[1] The field of AI research was founded at a workshop held on the campus of Dartmouth College, USA during the summer of 1956.[2] Those who attended would become the leaders of AI research for decades. Many of them predicted that a machine as intel

In [46]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "5": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [47]:
NUMBER=5 
SUBJECT="artificial intelligence"
TONE="simple"

In [48]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The history of artificial intelligence (AI) began in antiquity, with myths, stories and rumors of artificial beings endowed with intelligence or consciousness by master craftsmen. The seeds of modern AI were planted by philosophers who attempted to describe the process of human thinking as the mechanical manipulation of symbols. This work culminated in the invention of the programmable digital computer in the 1940s, a machine based on the abstract essence of mathematical reasoning. This device and the ideas behind it inspired a handful of scientists to begin seriously discussing the possibility of building an electronic brain.

Alan Turing was the first person to carry out substantial research in the field that he called Machine Intelligence.[1] The field of AI research was founded at a workshop held on the campus of Dartmouth College, USA during the summer of 1956.[2] Those who 

In [49]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2166
Prompt Tokens:1448
Completion Tokens:718
Total Cost:0.003608


In [50]:
response

{'text': 'The history of artificial intelligence (AI) began in antiquity, with myths, stories and rumors of artificial beings endowed with intelligence or consciousness by master craftsmen. The seeds of modern AI were planted by philosophers who attempted to describe the process of human thinking as the mechanical manipulation of symbols. This work culminated in the invention of the programmable digital computer in the 1940s, a machine based on the abstract essence of mathematical reasoning. This device and the ideas behind it inspired a handful of scientists to begin seriously discussing the possibility of building an electronic brain.\n\nAlan Turing was the first person to carry out substantial research in the field that he called Machine Intelligence.[1] The field of AI research was founded at a workshop held on the campus of Dartmouth College, USA during the summer of 1956.[2] Those who attended would become the leaders of AI research for decades. Many of them predicted that a mach

In [51]:
quiz=response.get("quiz")

In [52]:
quiz=json.loads(quiz)

In [53]:
quiz

{'1': {'mcq': 'Who is considered the first person to carry out substantial research in the field of artificial intelligence?',
  'options': {'a': 'Alan Turing',
   'b': 'James Lighthill',
   'c': 'Dartmouth College',
   'd': 'Japanese Government'},
  'correct': 'a'},
 '2': {'mcq': 'When and where was the field of AI research founded?',
  'options': {'a': 'In the 1940s at Dartmouth College',
   'b': 'In the 1950s at Dartmouth College',
   'c': 'In the 1940s at MIT',
   'd': 'In the 1950s at MIT'},
  'correct': 'b'},
 '3': {'mcq': 'What event led to the funding for undirected research into artificial intelligence being stopped?',
  'options': {'a': 'Criticism from James Lighthill',
   'b': 'Ongoing pressure from congress',
   'c': 'Both a and b',
   'd': 'None of the above'},
  'correct': 'c'},
 '4': {'mcq': 'What term is used to describe the period of decreased funding and interest in AI after the 1970s?',
  'options': {'a': 'AI winter',
   'b': 'AI summer',
   'c': 'AI spring',
   'd':

In [55]:
review = json.loads(response.get("review"))
review

{'1': {'mcq': 'What is considered the birth of AI research?',
  'options': {'a': 'The invention of the programmable digital computer',
   'b': 'The workshop at Dartmouth College',
   'c': 'The research by Alan Turing',
   'd': 'The funding by the U.S. and British Governments'},
  'correct': 'b'},
 '2': {'mcq': 'What term did Alan Turing use to describe his research in AI?',
  'options': {'a': 'Machine Consciousness',
   'b': 'Artificial Beings',
   'c': 'Machine Intelligence',
   'd': 'Symbol Manipulation'},
  'correct': 'c'},
 '3': {'mcq': 'What happened in response to the criticism from James Lighthill and ongoing pressure from congress?',
  'options': {'a': 'The birth of AI research',
   'b': 'The funding for AI research stopped',
   'c': 'The development of the programmable digital computer',
   'd': 'The application of powerful computer hardware'},
  'correct': 'b'},
 '4': {'mcq': 'What is the term used to describe the period of reduced funding and interest in AI research?',
  'op

In [56]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [59]:
quiz_table_data

[{'MCQ': 'Who is considered the first person to carry out substantial research in the field of artificial intelligence?',
  'Choices': 'a: Alan Turing | b: James Lighthill | c: Dartmouth College | d: Japanese Government',
  'Correct': 'a'},
 {'MCQ': 'When and where was the field of AI research founded?',
  'Choices': 'a: In the 1940s at Dartmouth College | b: In the 1950s at Dartmouth College | c: In the 1940s at MIT | d: In the 1950s at MIT',
  'Correct': 'b'},
 {'MCQ': 'What event led to the funding for undirected research into artificial intelligence being stopped?',
  'Choices': 'a: Criticism from James Lighthill | b: Ongoing pressure from congress | c: Both a and b | d: None of the above',
  'Correct': 'c'},
 {'MCQ': 'What term is used to describe the period of decreased funding and interest in AI after the 1970s?',
  'Choices': 'a: AI winter | b: AI summer | c: AI spring | d: AI fall',
  'Correct': 'a'},
 {'MCQ': 'When did investment and interest in AI boom again?',
  'Choices': 

In [57]:
quiz=pd.DataFrame(quiz_table_data)

In [58]:
quiz.to_csv("AI_quiz.csv",index=False)

In [60]:
review_table_data = []
for key, value in review.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    review_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [61]:
review_table_data

[{'MCQ': 'What is considered the birth of AI research?',
  'Choices': 'a: The invention of the programmable digital computer | b: The workshop at Dartmouth College | c: The research by Alan Turing | d: The funding by the U.S. and British Governments',
  'Correct': 'b'},
 {'MCQ': 'What term did Alan Turing use to describe his research in AI?',
  'Choices': 'a: Machine Consciousness | b: Artificial Beings | c: Machine Intelligence | d: Symbol Manipulation',
  'Correct': 'c'},
 {'MCQ': 'What happened in response to the criticism from James Lighthill and ongoing pressure from congress?',
  'Choices': 'a: The birth of AI research | b: The funding for AI research stopped | c: The development of the programmable digital computer | d: The application of powerful computer hardware',
  'Correct': 'b'},
 {'MCQ': 'What is the term used to describe the period of reduced funding and interest in AI research?',
  'Choices': 'a: AI Winter | b: AI Boom | c: AI Revolution | d: AI Renaissance',
  'Correct

In [62]:
review=pd.DataFrame(review_table_data)
review.to_csv("review.csv",index=False)

In [63]:
quiz

,MCQ,Choices,Correct
0,Who is considered the first person to carry ou...,a: Alan Turing | b: James Lighthill | c: Dartm...,a
1,When and where was the field of AI research fo...,a: In the 1940s at Dartmouth College | b: In t...,b
2,What event led to the funding for undirected r...,a: Criticism from James Lighthill | b: Ongoing...,c
3,What term is used to describe the period of de...,a: AI winter | b: AI summer | c: AI spring | d...,a
4,When did investment and interest in AI boom ag...,a: In the 1940s | b: In the 1970s | c: In the ...,d


In [64]:
review

,MCQ,Choices,Correct
0,What is considered the birth of AI research?,a: The invention of the programmable digital c...,b
1,What term did Alan Turing use to describe his ...,a: Machine Consciousness | b: Artificial Being...,c
2,What happened in response to the criticism fro...,a: The birth of AI research | b: The funding f...,b
3,What is the term used to describe the period o...,a: AI Winter | b: AI Boom | c: AI Revolution |...,a
4,What led to the boom in investment and interes...,a: The development of machine learning | b: Th...,a
